In [1]:
import psydac.core.interface as inter
import numpy as np
import sympy as sym
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import derivatives as der
import projectors as proj
import evaluation as eva
import integrate as integr
import time
import mass_matrices_curved as ma

In [2]:
# ... define boundary conditions in each direction (True: periodic, False: homogeneous Dirichlet)
bc = [False, False, False]
full = [True, True, True]

bc_x, bc_y, bc_z = bc

# ... length of computational domain in each direction
Lx = 1
Ly = 1
Lz = 1

# ... number of elements in each direction and corresponding element boundaries
Nel_x = 5
Nel_y = 5
Nel_z = 5

dx = Lx/Nel_x
dy = Ly/Nel_y
dz = Lz/Nel_z

el_b_x = np.linspace(0, Lx, Nel_x + 1)
el_b_y = np.linspace(0, Ly, Nel_y + 1)
el_b_z = np.linspace(0, Lz, Nel_z + 1)

# ... degree of spline basis (of the 0-forms) in each direction
p = [3, 3, 3]
px, py, pz = p

# ... number of basis functions
Nbase = [Nel_x + px, Nel_y + py, Nel_z + pz] 
Nbase_x, Nbase_y, Nbase_z = Nbase

# ... number of degrees of freedom
Nbase_x_0 = Nbase_x - bc_x*px - (1 - bc_x)*2
Nbase_y_0 = Nbase_y - bc_y*py - (1 - bc_y)*2
Nbase_z_0 = Nbase_z - bc_z*pz - (1 - bc_z)*2


# ... knot vectors
if bc_x == True:
    Tx = inter.make_periodic_knots(px, Nbase_x)
else:
    Tx = inter.make_open_knots(px, Nbase_x)
    
if bc_y == True:
    Ty = inter.make_periodic_knots(py, Nbase_y)
else:
    Ty = inter.make_open_knots(py, Nbase_y)
    
if bc_z == True:
    Tz = inter.make_periodic_knots(pz, Nbase_z)
else:
    Tz = inter.make_open_knots(pz, Nbase_z)
    
T = [Tx, Ty, Tz]

# ... define mapping
R1 = 0.2
R2 = 1.0


#gs = lambda q1, q2, q3 : 2*np.pi*(R2 - R1)*(q1*(R2 - R1) + R1)
g = lambda q1, q2, q3 : 1.

#Ginv = [[lambda q1, q2, q3 : 1/(R2 - R1)**2, lambda q1, q2, q3 : 0., lambda q1, q2, q3 : 0.], [lambda q1, q2, q3 : 0., lambda q1, q2, q3 : 1/(4*np.pi**2*(q1*(R2 - R1)**2 + R1)**2), lambda q1, q2, q3 : 0.], [lambda q1, q2, q3 : 0., lambda q1, q2, q3 : 0., lambda q1, q2, q3 : 1.]]
Ginv = [[lambda q1, q2, q3 : 1., lambda q1, q2, q3 : 0., lambda q1, q2, q3 : 0.], [lambda q1, q2, q3 : 0., lambda q1, q2, q3 : 1., lambda q1, q2, q3 : 0.], [lambda q1, q2, q3 : 0., lambda q1, q2, q3 : 0., lambda q1, q2, q3 : 1.]]

In [3]:
test1, mxx, mxy, mxz, myx, myy, myz, mzx, mzy, mzz = ma.mass_matrix_V1(p, Nbase, T, Ginv, g, [None, None, None])

In [4]:
test2 = integr.mass_matrix_V1(p, Nbase, T, bc, full)

In [5]:
np.allclose(test1.toarray(), test2.toarray())

True

In [9]:
myy.toarray()

array([[0.00734694, 0.0045    , 0.00094898, ..., 0.        , 0.        ,
        0.        ],
       [0.0045    , 0.01138776, 0.00803571, ..., 0.        , 0.        ,
        0.        ],
       [0.00094898, 0.00803571, 0.01680612, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.01680612, 0.00803571,
        0.00094898],
       [0.        , 0.        , 0.        , ..., 0.00803571, 0.01138776,
        0.0045    ],
       [0.        , 0.        , 0.        , ..., 0.00094898, 0.0045    ,
        0.00734694]])

In [8]:
test2[448:2*448, 448:2*448].toarray()

array([[0.00734694, 0.0045    , 0.00094898, ..., 0.        , 0.        ,
        0.        ],
       [0.0045    , 0.01138776, 0.00803571, ..., 0.        , 0.        ,
        0.        ],
       [0.00094898, 0.00803571, 0.01680612, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.01680612, 0.00803571,
        0.00094898],
       [0.        , 0.        , 0.        , ..., 0.00803571, 0.01138776,
        0.0045    ],
       [0.        , 0.        , 0.        , ..., 0.00094898, 0.0045    ,
        0.00734694]])

In [17]:
Ty[1:-1]

array([0. , 0. , 0. , 0.2, 0.4, 0.6, 0.8, 1. , 1. , 1. ])